In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../..')

import bayesbridge.reg_coef_sampler.hamiltonian_monte_carlo as hmc
from tests.distributions import BivariateGaussian, BivariateSkewNormal

## Run HMC on a bivariate Gaussian target.

In [ ]:
bivariate_gauss = BivariateGaussian(rho=.9, sigma=np.array([1., 2.]))
pc_var, _ = bivariate_gauss.get_principal_components()
pc_scale = np.sqrt(pc_var)

# Choose near-optimal stepsize and integration time.
dt = np.array([.7, .8]) * 2. * np.min(pc_scale)
n_step = np.ceil(
    np.array([.8, 1.]) * np.max(pc_scale) / np.max(dt)
)

n_burnin = 10
n_sample = 10 ** 5

theta0 = np.zeros(2)
f = bivariate_gauss.compute_logp_and_gradient
samples, logp_samples, accept_prob, time_elapsed \
    = hmc.generate_samples(f, theta0, dt, n_step, n_burnin, n_sample)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = [
    bivariate_gauss.sigma[0] * np.linspace(-4, 4, 51),
    bivariate_gauss.sigma[1] * np.linspace(-4, 4, 51)
]
marginal_pdf = [
    bivariate_gauss.compute_marginal_pdf(bins[axis], axis) for axis in [0, 1]
]

plt.subplot(2, 2, 1)
plt.hist2d(samples[0, :], samples[1, :],
           bins=bins, normed=1, cmap='inferno')
plt.ylabel(r'$\theta_2')
plt.xlabel(r'$\theta_1$')
plt.colorbar(ticks=[])

for axis in [0, 1]:
    plt.subplot(2, 2, 3 - axis)
    plt.hist(samples[axis, :], bins=bins[axis], normed=1)
    plt.plot(bins[axis], marginal_pdf[axis])
    plt.xlabel(r'$\theta_{:d}$'.format(axis + 1))

plt.tight_layout()
plt.show()

## Run HMC on a rotated 2-d skew normal.
Gaussians have many special features that makes them particularly simple for HMC, so additional sanity check with a non-Gaussian target won't hurt.

In [ ]:
skewnorm = BivariateSkewNormal()
f = skewnorm.compute_logp_and_gradient

In [ ]:
dt = np.array([.25, .3])
n_step = np.array([3, 6])
n_burnin = 10
n_sample = 5 * 10 ** 4

theta0 = np.zeros(2)
samples, logp_samples, accept_prob, time_elapsed \
    = hmc.generate_samples(f, theta0, dt, n_step, n_burnin, n_sample)

In [ ]:
plt.figure(figsize=(14, 10))
plt.rcParams['font.size'] = 20

bins = [
    np.linspace(-2, 4, 51),
    np.linspace(-3, 3, 51)
]
marginal_pdf = skewnorm.compute_marginal_pdf(*bins)

plt.subplot(2, 2, 1)
plt.hist2d(samples[0, :], samples[1, :],
           bins=bins, normed=1, cmap='inferno')
plt.ylabel(r'$\log_{10}(N)$')
plt.xlabel(r'$\log(p / (1 - p))$')
plt.colorbar(ticks=[])

for axis in [0, 1]:
    plt.subplot(2, 2, 3 - axis)
    plt.hist(samples[axis, :], bins=bins[axis], normed=1)
    plt.plot(bins[axis], marginal_pdf[axis])
    plt.xlabel(r'$\theta_{:d}$'.format(axis + 1))

plt.tight_layout()
plt.show()